**Note:** This is a DDL notebook. Run this only once

In [1]:
from pyspark.sql import SparkSession

from seed.unity import conf

spark: SparkSession = SparkSession.builder.config(conf=conf).getOrCreate()
print(f"Spark {spark.version} is up and running!")

:: loading settings :: url = jar:file:/opt/bitnami/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
io.unitycatalog#unitycatalog-spark_2.12 added as a dependency
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
software.amazon.awssdk#bundle added as a dependency
org.slf4j#slf4j-simple added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1c52c3ef-16bf-4506-9bf0-e681028a633c;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.3.1 in central
	found io.delta#delta-storage;3.3.1 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found io.unitycatalog#unitycatalog-spark_2.12;0.2.1 in central
	found io.unitycatalog#unitycatalog-client;0.2.1 in central
	found org.slf4j#slf4j-api;2.0.13 in central
	found org.apache.logging.log4j#log4j-slf4j2-impl;2.23.1 in central
	found o

Spark 3.5.5 is up and running!


In [2]:
spark.sql("SHOW CATALOGS").show()

# spark.sql("CREATE SCHEMA IF NOT EXISTS unity.dev")

# spark.sql("SHOW SCHEMAS FROM unity").show()

spark.sql("SHOW TABLES IN unity.default").show()

# spark.sql("SHOW TABLES IN unity.dev").show()

# spark.sql("SELECT * FROM default.marksheet LIMIT 5;").show()

+-------------+
|      catalog|
+-------------+
|spark_catalog|
|        unity|
+-------------+

+---------+-----------------+-----------+
|namespace|        tableName|isTemporary|
+---------+-----------------+-----------+
|  default|        marksheet|      false|
|  default|marksheet_uniform|      false|
|  default|            names|      false|
|  default|          numbers|      false|
|  default|             test|      false|
|  default|   user_countries|      false|
+---------+-----------------+-----------+



In [8]:
# Read flights data from parquet file

df = spark.read.parquet("s3a://seed/flights-1m.parquet")
df.show(5)

df.printSchema()

df.createOrReplaceTempView("raw_flights")

spark.sql("""
    SELECT
        MIN(FL_DATE) AS min_date,
        MAX(FL_DATE) AS max_date,
        COUNT(*) AS num_rows
    FROM raw_flights;
""").show()

spark.sql("""
    SELECT
        FL_DATE,
        COUNT(*) AS num_rows
    FROM raw_flights
    GROUP BY FL_DATE
    ORDER BY FL_DATE
    LIMIT 5;
""").show()

+----------+---------+---------+--------+--------+---------+---------+
|   FL_DATE|DEP_DELAY|ARR_DELAY|AIR_TIME|DISTANCE| DEP_TIME| ARR_TIME|
+----------+---------+---------+--------+--------+---------+---------+
|2006-01-01|        5|       19|     350|    2475| 9.083333|12.483334|
|2006-01-02|      167|      216|     343|    2475|11.783334|15.766666|
|2006-01-03|       -7|       -2|     344|    2475| 8.883333|12.133333|
|2006-01-04|       -5|      -13|     331|    2475| 8.916667|    11.95|
|2006-01-05|       -3|      -17|     321|    2475|     8.95|11.883333|
+----------+---------+---------+--------+--------+---------+---------+
only showing top 5 rows

root
 |-- FL_DATE: date (nullable = true)
 |-- DEP_DELAY: short (nullable = true)
 |-- ARR_DELAY: short (nullable = true)
 |-- AIR_TIME: short (nullable = true)
 |-- DISTANCE: short (nullable = true)
 |-- DEP_TIME: float (nullable = true)
 |-- ARR_TIME: float (nullable = true)

+----------+----------+--------+
|  min_date|  max_date|n

In [ ]:
# Create flights table from parquet file

spark.sql("""
    CREATE TABLE IF NOT EXISTS dev.flights
    USING delta
    LOCATION 's3a://iceberg/'
    AS
    SELECT
        distance,
        dep_time,
        arr_time
    FROM raw_flights;
""")

spark.sql("SELECT * FROM dev.flights LIMIT 5;").show()

In [ ]:
spark.stop()